# Setup

In [40]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# Generate the data

In [6]:
class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            # np.argmax取一个vocabulary的概率分布最大的那个
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)

In [137]:
# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    # int后将001这样的字符转成了1
    # f用于随机生成N位数，N<=(DIGITS=3)
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            # range(1)或2,3
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))

Generating data...
Total questions: 50000


# Vectorize the data

In [140]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


# Build the model

In [141]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense (Dense)                (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Train the model

In [142]:
epochs = 30
batch_size = 64


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val)
    )
    # Select 10 samples from the validation set at random so we can visualize errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        # 随机取出x_val中的第ind组
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 1
704/704 [==============================] - 7s 9ms/step - loss: 1.8283 - accuracy: 0.3359 - val_loss: 1.6998 - val_accuracy: 0.3761
Q 711+670 T 1381 ☒ 1011
Q 102+1   T 103  ☒ 11  
Q 75+730  T 805  ☒ 705 
Q 60+655  T 715  ☒ 665 
Q 52+29   T 81   ☒ 11  
Q 58+853  T 911  ☒ 805 
Q 8+137   T 145  ☒ 11  
Q 570+199 T 769  ☒ 101 
Q 324+664 T 988  ☒ 103 
Q 69+173  T 242  ☒ 701 

Iteration 2
704/704 [==============================] - 6s 8ms/step - loss: 1.5793 - accuracy: 0.4101 - val_loss: 1.4456 - val_accuracy: 0.4601
Q 36+168  T 204  ☒ 211 
Q 220+9   T 229  ☒ 221 
Q 620+38  T 658  ☒ 661 
Q 978+12  T 990  ☒ 981 
Q 959+240 T 1199 ☒ 1101
Q 186+872 T 1058 ☒ 101 
Q 639+44  T 683  ☒ 618 
Q 5+319   T 324  ☒ 431 
Q 48+62   T 110  ☒ 100 
Q 717+466 T 1183 ☒ 1178

Iteration 3
704/704 [==============================] - 6s 8ms/step - loss: 1.3339 - accuracy: 0.5006 - val_loss: 1.2465 - val_accuracy: 0.5330
Q 452+984 T 1436 ☒ 1434
Q 208+48  T 256  ☒ 281 
Q 745+71  T 816  ☒ 810 
Q 450+680 T 1130

Q 341+83  T 424  ☑ 424 
Q 4+488   T 492  ☑ 492 
Q 44+72   T 116  ☑ 116 
Q 135+31  T 166  ☑ 166 
Q 66+841  T 907  ☑ 907 
Q 0+259   T 259  ☑ 259 
Q 99+315  T 414  ☑ 414 
Q 86+708  T 794  ☑ 794 
Q 2+810   T 812  ☑ 812 
Q 371+97  T 468  ☑ 468 

Iteration 23
704/704 [==============================] - 6s 8ms/step - loss: 0.0211 - accuracy: 0.9955 - val_loss: 0.0754 - val_accuracy: 0.9748
Q 45+795  T 840  ☑ 840 
Q 944+9   T 953  ☑ 953 
Q 126+6   T 132  ☑ 132 
Q 983+28  T 1011 ☑ 1011
Q 405+861 T 1266 ☑ 1266
Q 813+88  T 901  ☑ 901 
Q 389+955 T 1344 ☒ 1354
Q 13+130  T 143  ☑ 143 
Q 814+672 T 1486 ☑ 1486
Q 2+688   T 690  ☑ 690 

Iteration 24
704/704 [==============================] - 6s 9ms/step - loss: 0.0202 - accuracy: 0.9953 - val_loss: 0.0100 - val_accuracy: 0.9987
Q 855+863 T 1718 ☑ 1718
Q 163+7   T 170  ☑ 170 
Q 364+88  T 452  ☑ 452 
Q 312+736 T 1048 ☑ 1048
Q 68+394  T 462  ☑ 462 
Q 91+690  T 781  ☑ 781 
Q 6+441   T 447  ☑ 447 
Q 86+327  T 413  ☑ 413 
Q 25+937  T 962  ☑ 962 
Q 477+73  T 55

In [ ]:
for i in range(10):
    ind = np.random.randint(0, len(x_val))
    # 随机取出x_val中的第ind组
    rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
    preds = np.argmax(model.predict(rowx), axis=-1)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    print("Q", q[::-1] if REVERSE else q, end=" ")
    print("T", correct, end=" ")
    if correct == guess:
        print("☑ " + guess)
    else:
        print("☒ " + guess)

In [155]:
questions = [' 543+21']
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)

preds = np.argmax(model.predict(x), axis=-1)
guess = ctable.decode(preds[0], calc_argmax=False)
guess

In [157]:
preds = np.argmax(model.predict(x), axis=-1)
guess = ctable.decode(preds[0], calc_argmax=False)
guess

'357 '

You'll get to 99+% validation accuracy after ~30 epochs.